In [1]:
import cv2, os
import numpy as np
import Model, Preprocess

from config import root, dir_resources

import warnings
warnings.filterwarnings("ignore")

In [2]:
model_name = 'v1'
model = Model.load_model( os.path.join(root, 'Model', model_name) )

Successfully loaded model from ../Model\v1


In [3]:
cascade_name = 'haarcascade_frontalface_default.xml'
face_cascade = cv2.CascadeClassifier( os.path.join(dir_resources, 'haarcascade', cascade_name) )

In [4]:
def draw_results(image_bgr, results):
    result_shape = (512, 512)

    extend_result_x = 300
    start_x = 520
    start_y = 20
    pos_y_dif = 20
    section_y_dif = 30

    font_face = cv2.FONT_HERSHEY_SIMPLEX
    font_scale = 0.45
    font_thickness = 1
    
    title_color = (255,255,255)
    
    name_add_x = 0
    score_add_x = 100

    line_add_x = 150
    line_lenght = 100
    line_thickness = 2
    
    from config import facial_expression_color_bgr as face_color
    from config import facial_expression as face_name
    
    image = image_bgr.copy()
    image = cv2.resize(image, result_shape)
    
    black_box = np.zeros((image.shape[0], extend_result_x, 3), dtype='uint8')
    image = np.hstack((image, black_box))
    
    list_of_data = []
    for result in results:
        data = []
        for i, score in enumerate(result):
            datum = (score, face_name[i], face_color[i])
            data.append(datum)
        data.sort(reverse=True)
        list_of_data.append(data)
        
    pos_x = start_x
    pos_y = start_y
    
    for i, data in enumerate(list_of_data):
        text_title = 'Index %d'%i
        cv2.putText(image, text_title, (pos_x,pos_y), fontFace=font_face, fontScale=font_scale, color=title_color, thickness=font_thickness)
        pos_y+=pos_y_dif
        for score, name, color in data:
            text_name = "%-10s"%name
            cv2.putText(image, text_name, (pos_x + name_add_x,pos_y), fontFace=font_face, fontScale=font_scale, color=color, thickness=font_thickness)
            
            text_score = "%4s%%"%('%.1f'%(score*100))
            cv2.putText(image, text_score, (pos_x + score_add_x,pos_y), fontFace=font_face, fontScale=font_scale, color=color, thickness=font_thickness)
            
            start_point = (int(pos_x + line_add_x), int(pos_y-5))
            end_point = (int(start_point[0] + line_lenght * score), int(start_point[1]))
            
            cv2.line(image, start_point, end_point, color=color, thickness=line_thickness)
            pos_y += pos_y_dif
            
        pos_y += section_y_dif
    
    return image

def draw_bounding_boxes(image_bgr, bounding_boxes):
    
    color = (255,0,0)
    
    bounding_box_thickness = 2
    
    font_face = cv2.FONT_HERSHEY_SIMPLEX
    font_scale = 0.45
    font_thickness = 1
    
    title_add_x = 0
    title_add_y = -10
    
    image = image_bgr.copy()
    
    for i, (x,y,w,h) in enumerate(bounding_boxes):
        cv2.rectangle(image, (x,y), (x+w, y+h), color, bounding_box_thickness)
        
        title = 'Index %d'%i
        cv2.putText(image, title, (x + title_add_x, y + title_add_y), fontFace=font_face, fontScale=font_scale, color=color, thickness=font_thickness)
    
    return image    

In [5]:
def predict_image(image_bgr):
    image_color = image_bgr.copy()
    image_gray = cv2.cvtColor(image_color, cv2.COLOR_BGR2GRAY)
    image_gray = cv2.equalizeHist(image_gray)
        
    images = []
    
    bounding_boxes = face_cascade.detectMultiScale(image_gray)
    for (x,y,w,h) in bounding_boxes:
        image = image_gray[x:x+w, y:y+h]
        images.append(image)
    
    images = [Preprocess.preprocess_gray(image) for image in images]
    images = np.asarray(images, 'float32')
    
    results = []
    if(len(images) > 0):
        results = model(images).numpy()
    
    result_image = image_color.copy()
    result_image = draw_bounding_boxes(result_image, bounding_boxes)
    result_image = draw_results(result_image, results)
    
    return result_image

def query_image(source, target):
    image = cv2.imread(source)
    result_image = predict_image(image)
    cv2.imwrite(target, result_image)
    
def query_video(source, target):
    print('Reading video from %s'%source, end='\r')
    cap = cv2.VideoCapture(source)
    assert(cap.isOpened())
    frames = []
    fps = cap.get(cv2.CAP_PROP_FPS)
    while(cap.isOpened()):
        status, frame = cap.read()
        if(status):
            frame = predict_image(frame)
            frames.append(frame)
        else:
            break
    cap.release()
    
    height, width = frames[0].shape[:2]
    
    out = cv2.VideoWriter(target, cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))
    for frame in frames:
        out.write(frame)
    out.release()
    print('Successfully saved video to %s'%target)
    

In [6]:
query_video( os.path.join(dir_resources,'video/sample_video.mp4'),  os.path.join(dir_resources,'video/sample_result_video.mkv'))

Successfully saved video to ../Resources/video/sample_result_video.mkv
